In [1]:
import os, random, sys
import numpy as np
import pandas as pd
from functools import partial
import pandas as pd
sys.path.insert(1, '../../../utils')
from regressionDataGen import load_openml_data

In [10]:
load_openml_data('.', 42351)

(      0  1   2  3    4  5  6  7        8         9  ... 23 24  25  26  27 28  \
 203  GP  M  17  R  GT3  T  1  2  at_home     other  ...  3  1   3   1   5  3   
 498  MS  F  16  U  GT3  T  2  1    other  services  ...  5  3   3   1   1  1   
 51   GP  F  15  U  LE3  T  4  2   health     other  ...  4  3   3   1   1  5   
 513  MS  F  16  U  GT3  T  3  1    other     other  ...  3  1   3   1   3  1   
 13   GP  M  15  U  GT3  T  4  3  teacher     other  ...  5  4   3   1   2  3   
 ..   .. ..  .. ..  ... .. .. ..      ...       ...  ... .. ..  ..  ..  .. ..   
 634  MS  F  18  U  GT3  T  3  3    other     other  ...  4  1   3   1   2  1   
 0    GP  F  18  U  GT3  A  4  4  at_home   teacher  ...  4  3   4   1   1  3   
 368  GP  M  18  U  LE3  T  4  4    other     other  ...  4  2   5   3   4  5   
 156  GP  F  15  U  GT3  T  1  1    other  services  ...  4  4   2   1   2  5   
 117  GP  M  16  U  GT3  T  3  3    other  services  ...  5  4   2   1   1  5   
 
     29  30  31   y  
 203

In [2]:
data = np.load('openml_datasets/42351_False.npy', allow_pickle=True)

In [3]:
X, y = data.item()['X'], data.item()['y']
attribute_names = data.item()['attr']
label =  data.item()['label']

In [4]:
X.shape

(649, 32)

In [4]:
df = pd.DataFrame(data = X)
df.columns = attribute_names
df['y'] = y

In [5]:
df.to_csv('student.csv')

In [6]:
np.random.seed(123)
df = df.sample(frac=1, random_state = 123)
n = df.shape[0]
n_train, n_valid, n_test = int(n*0.7), int(n*.15), int(n*.15)
train_idx, valid_idx, test_idx = np.arange(n_train), np.arange(n_train, n_train+n_valid), np.arange(n_train+n_valid, n_train+n_valid+n_test)

In [7]:
train_df, valid_df, test_df = df.iloc[train_idx].reset_index(drop = True), df.iloc[valid_idx].reset_index(drop = True), df.iloc[test_idx].reset_index(drop = True)
train_df.to_csv('student_train_full.csv')
valid_df.to_csv('student_valid.csv')
test_df.to_csv('student_test.csv')

In [55]:
df['failures']

376    0
142    0
43     0
162    0
351    2
      ..
98     0
322    1
382    0
365    0
510    0
Name: failures, Length: 649, dtype: uint8

In [80]:
def studentConverter(row, label = True, context = True):
    if context:
        she = 'He' if row['sex'] == 'M' else 'She'
        her = 'His' if row['sex'] == 'M' else 'Her'
        him = 'Him' if row['sex'] == 'M' else 'Her'
        gender = 'male' if row['sex'] == 'M' else 'female' 
        school = 'Gabriel Pereira' if row['school'] == 'GP' else 'Mousinho da Silveira'
        fs = 'less or equal to 3 members' if row['famsize'] == 'LE3' else 'greater than 3 members'
        pstatus = 'together' if row['Pstatus'] == 'T' else 'apart'
        address = 'urban' if row['address'] == 'U' else 'rural'
        
        def education(x):
            if x == '0':
                return ' is uneducated'
            elif x == '1':
                return "'s highest education is 4th grade"
            elif x == '2':
                return "'s highest education is 5th-9th grade"
            elif x == '3':
                return ' had a secondary education'
            else:
                return ' is highly educated'
        
        def job(x):
            if x == 'at_home':
                return ', and now is working at home'
            elif x == 'health':
                return ', and now is working on healthcare-related field'
            elif x == 'services':
                return ', and now is working as a civil service employee'
            elif x == 'teacher':
                return ', and now is working as a teacher'
            else:
                return ''
            
        def failure(x):
            if x == 4:
                return 'failed more than 4 courses. '
            else:
                return 'failed %d courses. '% x
            
        def select(x):
            if x == 'yes':
                return ''
            else:
                return 'no '
            
        def score(x, good = True):
            if good:
                adj_pos = 'good'
                adj_neg = 'bad'
            else:
                adj_pos = 'high'
                adj_neg = 'low'
            if x == 1:
                return 'very %s' % adj_neg
            elif x == 2:
                return adj_neg
            elif x == 3:
                return 'medium'
            elif x == 4:
                return adj_pos
            else:
                return 'very %s' % adj_pos
        
        if not row['guardian'] in ['father', 'mother']:
            guardian = '%s %s' % (her.lower(), row['guardian'])
        else:
            guardian = 'neither %s mother nor father' % (her.lower())
            
        if row['reason'] == 'home':
            reason = '%s chose this school because it is close to home. ' % she
        elif row['reason'] == 'reputation':
            reason = '%s chose this school for its reputation. ' % she
        elif row['reason'] == 'course':
            reason = '%s chose this school because of its courses. ' % she
        else:
            reason = ''
            
        nursery = 'has attended' if row['nursery'] else 'has never attended'
        higher = 'wants' if row['higher'] else 'does not want'
        
        s1 = 'A %d-year-old %s student from %s school, is living in %s area. ' % (row['age'], gender, school, address)
        s2 = '%s family has %s, and %s parents are living %s. ' % (her, fs, her.lower(), pstatus)
        s3 = '%s mother%s%s. %s father%s%s. ' % (her, education(row['Medu']), job(row['Mjob']), her, education(row['Fedu']), job(row['Fjob']))
        s4 = '%s guardian is %s. ' % (her, guardian)
        s5 = reason
        s6 = 'It takes %s %d hours to get to school. ' % (him.lower(), row['traveltime'])
        s7 = '%s studies %d hours every week, and %s' % (she, row['studytime'], failure(row['failures']))
        s8 = '%s has %sextra education support, has %sfamily education support, has %sextra paid classes within the course subject, has %sextra curricular activities. ' % (she, select(row['schoolsup']), select(row['famsup']), select(row['paid']), select(row['activities'])) 
        s9 = '%s %s nursery school, and %s to take higher education. ' % (she, nursery, higher)
        s10 = '%s has %sinternet access at home, with %sromantic relationship. ' % (she, select(row['internet']), select(row['romantic']))
        s11 = '%s family relationships are %s. ' % (her, score(row['famrel']))
        s12 = '%s has %s free time after school, and goes out with friends in %s frequency. ' % (she, score(row['freetime'], False), score(row['goout'], False))
        s13 = '%s workday alcohol consumptions is %s, and weekend alcohol consumption is %s. ' % (her, score(row['Dalc'], False), score(row['Walc'], False))
        s14 = '%s is a %s health status, and has %d number of school absences. ' % (she, score(row['health']), row['absences'])
        s15 = '%s first period grade is %d, and second period grade is %d. What is %s final grade? ' % (her, row['G1'], row['G2'], her.lower())
        prompt = s1+s2+s3+s4+s5+s6+s7+s8+s9+s10+s11+s12+s13+s14+s15
        if label:
            completion = row['y']
            return "{\"prompt\":\"%s###\", \"completion\":\"%s@@@\"}" % (prompt, completion)
        else:
            return prompt
    else:
        s = 'When we have '
        for i in range(len(row)-1):
            s += 'x%d=%s, ' % (i+1, row.iloc[i])
        s += 'what is the y?'
        if label:
            completion = row['y']
            return "{\"prompt\":\"%s###\", \"completion\":\"%s@@@\"}" % (s, completion)
        else:
            return s

In [81]:
jsonl = '\n'.join(test_df.apply(func = studentConverter, axis = 1).tolist())
with open('student_fn_test.jsonl', 'w') as outfile:
    outfile.write(jsonl)
jsonl = '\n'.join(test_df.apply(func = partial(studentConverter, context = False), axis = 1).tolist())
with open('student_test.jsonl', 'w') as outfile:
    outfile.write(jsonl)
jsonl = '\n'.join(valid_df.apply(func = studentConverter, axis = 1).tolist())
with open('student_fn_valid.jsonl', 'w') as outfile:
    outfile.write(jsonl)
jsonl = '\n'.join(valid_df.apply(func = partial(studentConverter, context = False), axis = 1).tolist())
with open('student_valid.jsonl', 'w') as outfile:
    outfile.write(jsonl)
jsonl = '\n'.join(train_df.apply(func = studentConverter, axis = 1).tolist())
with open('student_fn_full_train.jsonl', 'w') as outfile:
    outfile.write(jsonl)
jsonl = '\n'.join(train_df.apply(func = partial(studentConverter, context = False), axis = 1).tolist())
with open('student_full_train.jsonl', 'w') as outfile:
    outfile.write(jsonl)

In [82]:
train_df_20 = train_df.sample(frac=0.2, random_state = 123).reset_index(drop = True)
train_df_40 = train_df.sample(frac=0.4, random_state = 123).reset_index(drop = True)
train_df_60 = train_df.sample(frac=0.6, random_state = 123).reset_index(drop = True)
train_df_80 = train_df.sample(frac=0.8, random_state = 123).reset_index(drop = True)

In [83]:
train_df_20.to_csv('student_train_20.csv')
train_df_40.to_csv('student_train_40.csv')
train_df_60.to_csv('student_train_60.csv')
train_df_80.to_csv('student_train_80.csv')

In [84]:
jsonl = '\n'.join(train_df_20.apply(func = studentConverter, axis = 1).tolist())
with open('student_fn_20_train.jsonl', 'w') as outfile:
    outfile.write(jsonl)
jsonl = '\n'.join(train_df_20.apply(func = partial(studentConverter, context = False), axis = 1).tolist())
with open('student_20_train.jsonl', 'w') as outfile:
    outfile.write(jsonl)
jsonl = '\n'.join(train_df_40.apply(func = studentConverter, axis = 1).tolist())
with open('student_fn_40_train.jsonl', 'w') as outfile:
    outfile.write(jsonl)
jsonl = '\n'.join(train_df_40.apply(func = partial(studentConverter, context = False), axis = 1).tolist())
with open('student_40_train.jsonl', 'w') as outfile:
    outfile.write(jsonl)
jsonl = '\n'.join(train_df_60.apply(func = studentConverter, axis = 1).tolist())
with open('student_fn_60_train.jsonl', 'w') as outfile:
    outfile.write(jsonl)
jsonl = '\n'.join(train_df_60.apply(func = partial(studentConverter, context = False), axis = 1).tolist())
with open('student_60_train.jsonl', 'w') as outfile:
    outfile.write(jsonl)
jsonl = '\n'.join(train_df_80.apply(func = studentConverter, axis = 1).tolist())
with open('student_fn_80_train.jsonl', 'w') as outfile:
    outfile.write(jsonl)
jsonl = '\n'.join(train_df_80.apply(func = partial(studentConverter, context = False), axis = 1).tolist())
with open('student_80_train.jsonl', 'w') as outfile:
    outfile.write(jsonl)